In [1]:
! pip install jmespath

In [2]:
import json
import os
import sys
import jmespath
import pandas as pd


#load json file
def load_json(file):
    with open(file, 'r') as f:
        return json.load(f)
    
#change the json file names
posts_id = load_json("daisychoii_posts.json")

posts_id.keys()

mattrife_keylist = list(posts_id.keys())

# posts_id[mattrife_keylist[0]] #still includes metadata
POST_NUM = 0
posts_id[mattrife_keylist[POST_NUM]]['items'][0]
# ^ This is the actual post info

post0_json = posts_id[mattrife_keylist[POST_NUM]]['items'][0]

post0 = {}

post0['id'] = post0_json['id']
post0['shortcode'] = post0_json['code'] #shortcode id
post0['caption'] = post0_json['caption']['text'] #caption

#We're interested in only the first (display) image
post0['head_image_url'] = post0_json['image_versions2']['candidates'][0]['url']

#Also interested in music metadata
music_metadata = post0_json['music_metadata']
post0_json['music_metadata']

post0['audio_type'] = music_metadata['audio_type']
music_metadata['audio_type'] == 'licensed_music'
#We want that to be true

music_info = music_metadata['music_info']

def parse_post_json(data: dict) -> dict:
    """Parse json data from instagram post and return a dictionary of relevant information"""
    result = jmespath.search(
        """{
        id: id,
        shortcode: code,
        caption: caption.text,
        head_image_url: image_versions2.candidates[0].url,
        audio_type: music_metadata.audio_type,
        music_title: music_metadata.music_info.music_asset_info.title,
        music_artist: music_metadata.music_info.music_asset_info.display_artist
    }""",
        data,
    )
    return result

parse_post_json(post0_json)

# for all posts, parse them, and save them to a new json file
all_posts = []
for post in posts_id.values():
    all_posts.append(parse_post_json(post['items'][0]))

with open('mattrife_posts_parsed.json', 'w') as f:
    json.dump(all_posts, f, indent=4)

# alternatively, for all posts, save into a pandas DF, and then into a csv

df = pd.DataFrame(all_posts)
df.to_csv('mattrife_posts_parsed.csv', index=False)

# for all "None" on audio type, remove them
df = df[df['audio_type'] == 'licensed_music']
# remove id, caption, audio type, 
df = df.drop(columns=['id', 'caption', 'audio_type'])
#can you rename shortcode to id?
df = df.rename(columns={'shortcode': 'ID'})


In [15]:
df

,ID,head_image_url,music_title,music_artist
1,DFzJ4LQxlTw,https://scontent-lax3-2.cdninstagram.com/v/t51...,Dream,"Suzy, BAEKHYUN"
4,DCgGK3TxJPY,https://scontent-lax3-1.cdninstagram.com/v/t51...,Chanel,Frank Ocean
5,DCYuI0YAzB_,https://scontent-lax3-2.cdninstagram.com/v/t51...,What If I Go? (feat. Bonzai),Mura Masa
6,DC2lg1Bxl0l,https://scontent-lax3-1.cdninstagram.com/v/t51...,UP (KARINA Solo),aespa
8,C9jU_gqx4cO,https://scontent-lax3-2.cdninstagram.com/v/t51...,Cookie,NewJeans
9,C9ATcCThz0g,https://scontent-lax3-1.cdninstagram.com/v/t51...,Tongue Tied,GROUPLOVE
10,DAfVk7Ex9MY,https://scontent-lax3-1.cdninstagram.com/v/t51...,Once Upon a Dream,Invadable Harmony
11,DE34artRKWu,https://scontent-lax3-2.cdninstagram.com/v/t51...,pixie dust,Dreamy Kid
12,C8wjHPih5lt,https://scontent-lax3-1.cdninstagram.com/v/t51...,Hashirimizu,Yamafutatsu
13,C9_1WhXx7Ai,https://scontent-lax3-2.cdninstagram.com/v/t51...,No Surprises,Radiohead


In [14]:
# save to csv
df.to_csv('mattrife_posts_parsed.csv', index=False)
